## FUSIONAR DF Y AÑADIR LABELS

In [1]:
import pandas as pd

# Cargar los datasets
df_no_crypto = pd.read_csv('model_no_crypto_data.csv', usecols=['title'])  # Cambia 'texto' por el nombre real de la columna
df_crypto = pd.read_csv('model_crypto_data.csv', usecols=['title'])  # Cambia 'texto' por el nombre real de la columna

# Etiquetar los datos
df_no_crypto['label'] = 0
df_crypto['label'] = 1

# Renombrar la columna de texto en ambos datasets para que tengan el mismo nombre
df_no_crypto.rename(columns={'title': 'text'}, inplace=True)
df_crypto.rename(columns={'title': 'text'}, inplace=True)

# Combinar los dos datasets en uno solo
df = pd.concat([df_no_crypto, df_crypto])

# Manejar valores NaN: eliminar filas con NaN o rellenarlas con una cadena vacía
df.dropna(subset=['text'], inplace=True)

# Verificar que los datos estén combinados correctamente
print(df.head())


                                                text  label
0  The FAA ban on some batteries on planes — star...      0
1  It’s a tradition: New Year’s Day provides all-...      0
3  Many people try to organize a space, rather th...      0
4  The last time the U.S. started this well, they...      0
5  For over a month, Mitt Romney has been well be...      0


# BUENO

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


# Asegurarse de que X_train y X_test son listas de texto
X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), df['label'], test_size=0.2, random_state=42)

# Vectorización de textos
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Entrenamiento del modelo SVM
model = SVC(kernel='linear')  # Utilizando el kernel lineal
model.fit(X_train_tfidf, y_train)

# Predicción y evaluación
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Prueba con un nuevo texto
new_text = ["Investing in cryptos might not be as OverPowered"]
new_text_tfidf = vectorizer.transform(new_text)
prediction = model.predict(new_text_tfidf)
print("Predicción:", "Relacionado con criptomonedas" if prediction[0] == 1 else "No relacionado con criptomonedas")

Accuracy: 0.9991803758562146
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    110550
           1       1.00      0.99      0.99      9017

    accuracy                           1.00    119567
   macro avg       1.00      0.99      1.00    119567
weighted avg       1.00      1.00      1.00    119567

Predicción: Relacionado con criptomonedas


In [4]:
pip install pickle

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


## PREDICCIONES

In [7]:
import pandas as pd
import os

# Leer el archivo CSV con nuevos textos para hacer predicciones
input_file = 'C:/Users/joans/Desktop/BIG DATA M1/TFM/Scripts/crypto_news_sentiment.csv' 
df_nuevos_datos = pd.read_csv(input_file)

# Vectorizar los textos de la columna 'CLEAN TEXT'
X_nuevos_datos_tfidf = vectorizer.transform(df_nuevos_datos['CLEAN TEXT'].tolist())

# Hacer predicciones usando el modelo SVM que ya tienes entrenado
predicciones = model.predict(X_nuevos_datos_tfidf)

# Añadir las predicciones al DataFrame
df_nuevos_datos['prediccion'] = predicciones
df_nuevos_datos['prediccion'] = df_nuevos_datos['prediccion'].apply(lambda x: 1 if x == 1 else 0)

# Contar los totales
total_registros = df_nuevos_datos.shape[0]
registros_crypto = df_nuevos_datos[df_nuevos_datos['prediccion'] == 1].shape[0]
registros_no_crypto = df_nuevos_datos[df_nuevos_datos['prediccion'] == 0].shape[0]

# Filtrar los registros que están relacionados con criptomonedas (prediccion = 1)
df_crypto = df_nuevos_datos[df_nuevos_datos['prediccion'] == 1]

# Guardar solo los registros relacionados con criptomonedas en un archivo con el mismo nombre que el archivo original pero con _filtered
output_file = os.path.splitext(input_file)[0] + '_filtered.csv'
df_crypto.to_csv(output_file, index=False)

# Mostrar los conteos
print(f"Total de registros: {total_registros}")
print(f"Registros relacionados con criptomonedas (1): {registros_crypto}")
print(f"Registros no relacionados con criptomonedas (0): {registros_no_crypto}")
print(f"Registros filtrados guardados en '{output_file}'")


Total de registros: 544462
Registros relacionados con criptomonedas (1): 271107
Registros no relacionados con criptomonedas (0): 273355
Registros filtrados guardados en 'C:/Users/joans/Desktop/BIG DATA M1/TFM/Scripts/crypto_news_combined_filtered.csv'
